In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import sys
# package_dir = "../input/bert_pretrained"
# sys.path.append(package_dir)
import os
print(os.listdir("../input/nvidiaapex/repository/NVIDIA-apex-39e153a"))
print(os.listdir("../input"))
print(os.listdir("../input/bert-2-new"))
print(os.listdir("../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/"))
print(os.path.exists("../input/models"))
print(os.path.exists("../input/bert-2"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))
#print(os.listdir("../input/fasttext-crawl-300d-2m"))
# uncased_l-24_h-1024_a-16/
# Any results you write to the current directory are saved as output.

['csrc', 'setup.py', '.gitignore', 'examples', 'LICENSE', 'apex', 'tests', 'docs', 'README.md', '.nojekyll']
['bert-2-new', 'bert-2', 'bert-pretrained-models', 'bert094084', 'ppbert', 'bert094048', 'models', 'jigsaw-unintended-bias-in-toxicity-classification', 'nvidiaapex', 'bert-pretrained']
['bert_pytorch.bin', 'bert_config.json', 'pytorch_model.bin']
['bert_model.ckpt.index', 'bert_model.ckpt.data-00000-of-00001', 'bert_model.ckpt.meta', 'bert_config.json', 'vocab.txt']
True
True


In [2]:
# Installing Nvidia Apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

/opt/conda/lib/python3.6/site-packages/pip/_internal/commands/install.py:244: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-7zbe4o85
Created temporary directory: /tmp/pip-req-tracker-dxbxvej4
Created requirements tracker '/tmp/pip-req-tracker-dxbxvej4'
Created temporary directory: /tmp/pip-install-w7j2mhh2
Processing /kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
  Created temporary directory: /tmp/pip-req-build-uj677x9j
  Added file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a to build tracker '/tmp/pip-req-tracker-dxbxvej4'
    Running setup.py (path:/tmp/pip-req-build-uj677x9j/setup.py) egg_info for package from file:///kaggle/input/nvidiaapex/repository/NVIDIA-apex-39e153a
    Running command python setup.py egg_info
    torch.__version__  =  1.1.0
    running egg_info
    creating pip-egg

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
import shutil
from apex import amp
import shutil
tqdm.pandas()

In [4]:

device=torch.device('cuda')

In [5]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"
num_to_load=900000                        #Train size to match time limit
valid_size= 100000                          #Validation Size
TOXICITY_COLUMN = 'target'

In [6]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT
package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam, BertModel


/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
# Translate model from tensorflow to pytorch
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
# BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /kaggle/input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/L

'../working/bert_config.json'

In [8]:
os.listdir("../working")

['__output__.json',
 '__notebook__.ipynb',
 'pytorch_model.bin',
 'bert_config.json']

In [9]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig
bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'+'bert_config.json')
# bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'+'bert_config.json')


In [10]:
class CustomModel(nn.Module):
    
    def __init__(self, bert_path, config, aux_number):
        super().__init__()
        # 加载并冻结bert模型参数
        self.bert = BertModel.from_pretrained(bert_path)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.w1 = torch.nn.Linear(768, 1)
        self.w2 = torch.nn.Linear(768, aux_number)
    def forward(self, seqs, attention_mask=None, labels=None):
        _, seqs = self.bert(seqs, attention_mask=attention_mask, output_all_encoded_layers=False)
        seqs = self.dropout(seqs)
        res = self.w1(seqs)
        res_aux = self.w2(seqs)
        out = torch.cat((res, res_aux), dim=-1)
        
        return out

In [11]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [12]:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
# BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'

In [13]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
# train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(num_to_load+valid_size)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))[num_to_load:]

from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()



symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}
def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x
def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

# train_df['comment_text'] = train_df['comment_text'].progress_apply(lambda x:preprocess(x))
print('loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)

# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']
y_aux_train = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
# y_train = np.vstack([(train_df['target'].values>=0.5).astype(np.int),weights]).T
# train_df = train_df.drop(['comment_text'],axis=1)
# # convert target to 0,1
# train_df['target']=(train_df['target']>=0.5).astype(float)
# identity_columns = [
#     'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
#     'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
# Overall
weights = np.ones((len(train_df),)) / 4
# Subgroup
weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
# Background Negative, Subgroup Positive
weights +=(( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
(train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
loss_weight = 1.0 / weights.mean()

y_train = np.vstack([(train_df['target'].values>=0.5).astype(np.int),weights]).T
X = sequences  
y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float)
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), y_train_torch)

loaded 904874 records


In [14]:

X = sequences               
# y = train_df[y_columns].values
# y_aux = train_df
# X_val = sequences[num_to_load:]                
# y_val = train_df[y_columns].values[num_to_load:]


In [15]:
# test_df=train_df.tail(valid_size).copy()
# train_df=train_df.head(num_to_load)

In [16]:
# y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float)
# train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), y_train_torch)
# train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))


In [17]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [18]:
output_model_file = "bert_pytorch.bin"

lr=2e-5 * 0.001
batch_size = 32
accumulation_steps=1
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

class CustomLoadModel(nn.Module):
    
    def __init__(self, config, aux_number):
        super().__init__()
        # 加载并冻结bert模型参数
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.w1 = torch.nn.Linear(768, 1)
        self.w2 = torch.nn.Linear(768, aux_number)
    def forward(self, seqs, attention_mask=None, labels=None):
        _, seqs = self.bert(seqs, attention_mask=attention_mask, output_all_encoded_layers=False)
        seqs = self.dropout(seqs)
        res = self.w1(seqs)
        res_aux = self.w2(seqs)
        out = torch.cat((res, res_aux), dim=-1)
        
        return out
#训练版本
# model = CustomModel('../working', bert_config, 6)
# model.zero_grad()
# model = model.to(device)

#加载版本

# model = CustomLoadModel(bert_config,6)
# model.load_state_dict(torch.load("../input/bert-2-new/bert_pytorch.bin"))
# model.zero_grad()
# model.to(device)

# model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=7)
# model.zero_grad()
# model = model.to(device)

model = BertForSequenceClassification(bert_config,num_labels=7)
# model.load_state_dict(torch.load("../input/bert094084/results/bert_pytorch.bin"))
model.load_state_dict(torch.load("../input/bert-2-new/bert_pytorch.bin"))
model.zero_grad()
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    for i,(x_batch, y_batch) in tk0: 
        optimizer.zero_grad()
#         y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device))
        loss = custom_loss(y_pred, y_batch.to(device))
#         loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


torch.save(model.state_dict(), output_model_file)


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

In [19]:
# # Run validation
# # The following 2 lines are not needed but show how to download the model for prediction
# model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
# # model.load_state_dict(torch.load(output_model_file ))
# model.load_state_dict(torch.load("../input/bert_pretrained/bert_pytorch.bin"))
# model.to(device)
# for param in model.parameters():
#     param.requires_grad=False
# model.eval()
# valid_preds = np.zeros((len(X_val)))
# valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
# valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

# tk0 = tqdm_notebook(valid_loader)
# for i,(x_batch,)  in enumerate(tk0):
#     pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
#     valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()




In [20]:
# # From baseline kernel

# def calculate_overall_auc(df, model_name):
#     true_labels = df[TOXICITY_COLUMN]>0.5
#     predicted_labels = df[model_name]
#     return metrics.roc_auc_score(true_labels, predicted_labels)

# def power_mean(series, p):
#     total = sum(np.power(series, p))
#     return np.power(total / len(series), 1 / p)

# def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
#     bias_score = np.average([
#         power_mean(bias_df[SUBGROUP_AUC], POWER),
#         power_mean(bias_df[BPSN_AUC], POWER),
#         power_mean(bias_df[BNSP_AUC], POWER)
#     ])
#     return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



# SUBGROUP_AUC = 'subgroup_auc'
# BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
# BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

# def compute_auc(y_true, y_pred):
#     try:
#         return metrics.roc_auc_score(y_true, y_pred)
#     except ValueError:
#         return np.nan

# def compute_subgroup_auc(df, subgroup, label, model_name):
#     subgroup_examples = df[df[subgroup]>0.5]
#     return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

# def compute_bpsn_auc(df, subgroup, label, model_name):
#     """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
#     subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
#     non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
#     examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
#     return compute_auc(examples[label]>0.5, examples[model_name])

# def compute_bnsp_auc(df, subgroup, label, model_name):
#     """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
#     subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
#     non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
#     examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
#     return compute_auc(examples[label]>0.5, examples[model_name])

# def compute_bias_metrics_for_model(dataset,
#                                    subgroups,
#                                    model,
#                                    label_col,
#                                    include_asegs=False):
#     """Computes per-subgroup metrics for all subgroups and one model."""
#     records = []
#     for subgroup in subgroups:
#         record = {
#             'subgroup': subgroup,
#             'subgroup_size': len(dataset[dataset[subgroup]>0.5])
#         }
#         record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
#         record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
#         record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
#         records.append(record)
#     return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [21]:

# MODEL_NAME = 'model1'
# test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
# TOXICITY_COLUMN = 'target'
# bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
# bias_metrics_df
# get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))